#### 수집데이터 전처리

##### 전처리용 모듈 임포트

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('./1_다나와_무선청소기결과.xlsx')
df.tail()

,상품명,스펙목록,가격
2220,라미 LED차량용 무선 청소기,"차량용청소기 / 무선 / [흡입력] ~6,000Pa / [필터] 헤파필터 / 스테인...",69000
2221,정인일렉텍 큐브온 핸디형 무선 진공 청소기,"차량용청소기 / 무선 / [흡입력] 6,000Pa / [소비전력] 70W / [필터...",53990
2222,바이거 DEX-WC230,물걸레청소기 / 회전식 / 물걸레전용 / 무선 / [배터리] 사용시간: 1시간20분...,146700
2223,앤커 무선 핸디형 청소기 유피 홈백 H11,"차량용청소기 / 무선 / [흡입력] 5,500Pa / [소비전력] 70W / [필터...",86900
2224,대우어플라이언스 아쿠아젯 LP-LQ1001,물걸레청소기 / 회전식 / 물걸레전용 / 무선 / [배터리] 충전시간: 5시간 / ...,144150


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품명     2225 non-null   object
 1   스펙목록    2225 non-null   object
 2   가격      2225 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


In [4]:
# 제조사 상품명 [상품코드]
# split시 두번째 파라미터 숫자가 자를 횟수
df['상품명'][2224].split(' ', 1)

['대우어플라이언스', '아쿠아젯 LP-LQ1001']

In [5]:
# 회사명, 모델명 분리
comp_list, prod_list = [], []
count = 0

for title in df['상품명']: 
    temps = str(title).split(' ', 1)
    try:
        if len(temps) == 2: # 회사명/모델명만 있는 상품이 존재
            comp_list.append(temps[0])
            prod_list.append(temps[1])
        else:
            comp_list.append(temps[0])
            prod_list.append(temps[0])
    except:
        print(count)

    count += 1

In [6]:
cate_list, usetime_list, suction_list = [], [], []

for spec_data in df['스펙목록']:
    spec_list = spec_data.split(' / ')

    # 카테고리, 사용시간, 흡입력 만 추출
    category = spec_list[0]
    cate_list.append(category)

    time_count = 0   
    suction_count = 0 # 사용시간, 흡력이 있을때만 카운트

    for spec in spec_list:
        if '사용시간' in spec:
            usetime_value = str(spec).split(' ')[1].strip()
            time_count += 1
        if '흡입력' in spec:
            suction_value = str(spec).split(' ')[1].strip()        
            suction_count += 1

    if time_count == 0: # 사용시간 정보가 없음
        usetime_value = ''
    if suction_count == 0: # 흡입력에 대한 정보 없음
        suction_value = ''    

    usetime_list.append(usetime_value)
    suction_list.append(suction_value)

##### 사용시간, 흡입력 단위 통일, 수치화

- 사용시간 : 분, 시간, 시간 분 등 분류가 되어 있고, object64 --> 수치화 할 수 없음
- 흡입력 : W, AW, Pa 등으로 단위가 일치하지 않고, object64 --> 수치화 불가

In [7]:
suction_list

['220W',
 '250W',
 '210W',
 '25000Pa',
 '210W',
 '150AW',
 '220W',
 '200W',
 '250W',
 '220W',
 '250W',
 '250W',
 '220W',
 '280W',
 '220W',
 '210W',
 '',
 '22000Pa',
 '',
 '220W',
 '40000Pa',
 '220W',
 '220W',
 '35000Pa',
 '220W',
 '240AW',
 '210W',
 '220W',
 '',
 '250W',
 '200AW',
 '210W',
 '210W',
 '15W',
 '200W',
 '210W',
 '',
 '',
 '35000Pa',
 '50AW',
 '100AW',
 '151AW',
 '150AW',
 '40000Pa',
 '250W',
 '210W',
 '',
 '150AW',
 '',
 '185AW',
 '',
 '150W',
 '60W',
 '',
 '35AW',
 '150W',
 '60W',
 '210W',
 '150W',
 '150W',
 '12000Pa',
 '220W',
 '',
 '210W',
 '',
 '18000Pa',
 '',
 '280W',
 '35AW',
 '220W',
 '60W',
 '',
 '200W',
 '200W',
 '',
 '',
 '262AW',
 '',
 '',
 '280W',
 '',
 '',
 '',
 '30AW',
 '370W',
 '30000Pa',
 '240AW',
 '210W',
 '42000Pa',
 '200W',
 '',
 '',
 '25000Pa',
 '100AW',
 '',
 '220W',
 '12000Pa',
 '',
 '220W',
 '',
 '25000Pa',
 '16000Pa',
 '9000Pa',
 '200W',
 '',
 '30W',
 '30000Pa',
 '13000Pa',
 '250W',
 '50W',
 '',
 '15000Pa',
 '',
 '155AW',
 '220W',
 '',
 '',
 '',
 '2

In [8]:
# 사용시간 단위 분으로 변환함수
def convert_to_min(cur_time):
    hour, minute = 0, 0

    try:
        if '시간' in cur_time:
            hour = int(cur_time.split('시간')[0])
            
            if '분' in cur_time:
                minute = int(cur_time.split('시간')[-1].split('분')[0])
            else:
                minute = 0
        else:
            hour = 0
            minute = int(cur_time.split('분')[0])
        
        return hour*60 + minute
    except:
        return None

print(convert_to_min(usetime_list[16]))

None


In [9]:
usemin_list = []

for ctime in usetime_list:
    usemin_list.append(convert_to_min(ctime))

In [10]:
len(usemin_list)

2225

In [11]:
# 흡입력 단위 통일
def convert_suction(value):
    try:
        value = str(value).upper() # AW, W, Pa -> PA
        if 'AW' in value or 'W' in value:
            result = value.replace('A', '').replace('W', '').replace(',', '')
            result = int(result)
        elif 'PA' in value:
            result = value.replace('PA', '').replace(',', '')
            result = int(result)/100
        else:
            result = None
    except:
        result = None

    return result

In [12]:
convert_suction(suction_list[19])

220

In [13]:
new_suction_list = []

for power in suction_list:
    new_suction_list.append(convert_suction(power))

In [14]:
len(new_suction_list)

2225

##### 전처리 완료 자료 DF생성

In [15]:
df_product = pd.DataFrame()
df_product['카테고리'] = cate_list
df_product['회사명'] = comp_list
df_product['제품명'] = prod_list
df_product['가격'] = df['가격']
df_product['사용시간'] = usemin_list
df_product['흡입력'] = new_suction_list

In [16]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   카테고리    2225 non-null   object 
 1   회사명     2225 non-null   object 
 2   제품명     2225 non-null   object 
 3   가격      2225 non-null   object 
 4   사용시간    1687 non-null   float64
 5   흡입력     1121 non-null   float64
dtypes: float64(2), object(4)
memory usage: 104.4+ KB


In [17]:
## 가격이 object인 이유는 '일시품절' --> None
df_product[df_product['가격'] == '일시품절'] = None

In [18]:
df_product['카테고리'].value_counts()

카테고리
핸디스틱청소기              1034
물걸레청소기                357
차량용청소기                301
핸디청소기                 169
침구청소기                 107
욕실청소기                  69
업소용청소기                 45
핸디/스틱청소기+로봇청소기세트       41
스틱청소기                  32
스틱청소기+로봇청소기트            5
진공청소기                   4
스틱청소기+공기청정기세트           4
스팀청소기                   4
청소기+제습기                 3
핸디/스틱청소기+핸디/스틱청소기       3
핸디스틱청소기+공기청정기세트         2
침구/카펫청소기                2
청소기+공기청정기               2
물걸레청소기+스팀청소기            2
차량용선풍기                  1
로봇청소기                   1
핸디/스틱청소기+전기밥세트          1
핸디/스틱청소기+물걸레청소기세트       1
핸디/스틱청소기+업소용청소기         1
욕실+물걸레청소기               1
수영장청소기                  1
Name: count, dtype: int64

In [19]:
df_product_final = df_product[df_product['카테고리'].isin(['핸디스틱청소기'])]

In [20]:
df_product_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1034 entries, 0 to 1367
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   카테고리    1034 non-null   object 
 1   회사명     1034 non-null   object 
 2   제품명     1034 non-null   object 
 3   가격      1034 non-null   object 
 4   사용시간    899 non-null    float64
 5   흡입력     674 non-null    float64
dtypes: float64(2), object(4)
memory usage: 56.5+ KB


In [21]:
df_product_final = df_product_final.astype({'가격':'int64'}) # object64 --> int64 변경

In [22]:
df_product_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1034 entries, 0 to 1367
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   카테고리    1034 non-null   object 
 1   회사명     1034 non-null   object 
 2   제품명     1034 non-null   object 
 3   가격      1034 non-null   int64  
 4   사용시간    899 non-null    float64
 5   흡입력     674 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 56.5+ KB


In [23]:
df_product_final.to_excel('./3_다나와_핸디스틱청소기_결과.xlsx', index=False)